In [6]:
import sqlite3
import pandas as pd
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
import folium

# Connect to the SQLite database
conn = sqlite3.connect('D:\Coursera\Files\Files\home\coder\project\master.db')

# Define the SQL query
query = """
SELECT latitude, longitude, stars 
FROM business 
WHERE city = 'Santa Barbara' 
AND state = 'CA' 
AND categories LIKE '%Shopping%'
"""

# Execute the query and load the results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Define the clustering algorithms
kmeans = KMeans(n_clusters=3)
dbscan = DBSCAN(eps=0.3)
agg_clustering = AgglomerativeClustering(n_clusters=3)

# Apply the clustering algorithms to the data
df['kmeans_labels'] = kmeans.fit_predict(df[['latitude', 'longitude']])
df['dbscan_labels'] = dbscan.fit_predict(df[['latitude', 'longitude']])
df['agg_clustering_labels'] = agg_clustering.fit_predict(df[['latitude', 'longitude']])

# Define the colors for the clusters
colors = ['red', 'blue', 'green']

# Plot the clusters on the map
m = folium.Map(location=[34.4213, -119.6982], zoom_start=13)

for index, row in df.iterrows():
    folium.Marker([row['latitude'], row['longitude']], icon=folium.Icon(color=colors[int(row['kmeans_labels'])])).add_to(m)

m.save(r'D:\Coursera\Files\Files\home\coder\project\kmeans_map.html')

m = folium.Map(location=[34.4213, -119.6982], zoom_start=13)

for index, row in df.iterrows():
    if row['dbscan_labels'] != -1:  # DBSCAN labels noise as -1
        folium.Marker([row['latitude'], row['longitude']], icon=folium.Icon(color=colors[int(row['dbscan_labels'])])).add_to(m)

m.save(r'D:\Coursera\Files\Files\home\coder\project\dbscan_map.html')

m = folium.Map(location=[34.4213, -119.6982], zoom_start=13)

for index, row in df.iterrows():
    folium.Marker([row['latitude'], row['longitude']], icon=folium.Icon(color=colors[int(row['agg_clustering_labels'])])).add_to(m)

m.save(r'D:\Coursera\Files\Files\home\coder\project\agg_clustering_map.html')

c:\Users\surze\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


In [7]:
import sqlite3
import pandas as pd
from sklearn.cluster import KMeans
import folium

# Connect to the SQLite database
conn = sqlite3.connect('D:\Coursera\Files\Files\home\coder\project\master.db')

# Define the SQL query
query = """
SELECT b.latitude, b.longitude, b.stars, COUNT(r.review_id) as review_count, AVG(r.stars) as avg_review_stars
FROM business b
JOIN review r ON b.business_id = r.business_id
WHERE b.city = 'Santa Barbara' 
AND b.state = 'CA' 
AND b.categories LIKE '%Shopping%'
GROUP BY b.business_id
"""

# Execute the query and load the results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Define the clustering algorithm
kmeans = KMeans(n_clusters=3)

# Apply the clustering algorithm to the data
df['kmeans_labels'] = kmeans.fit_predict(df[['latitude', 'longitude', 'stars', 'review_count', 'avg_review_stars']])

# Define the colors for the clusters
colors = ['red', 'blue', 'green']

# Plot the clusters on the map
m = folium.Map(location=[34.4213, -119.6982], zoom_start=13)

for index, row in df.iterrows():
    folium.Marker([row['latitude'], row['longitude']], 
                  icon=folium.Icon(color=colors[int(row['kmeans_labels'])]), 
                  popup=f"Stars: {row['stars']}, Review Count: {row['review_count']}, Average Review Stars: {row['avg_review_stars']}").add_to(m)

m.save(r'D:\Coursera\Files\Files\home\coder\project\kmeans_map.html')

c:\Users\surze\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


In [8]:
# Calculate the mean of each feature for each cluster
cluster_profiles = df.groupby('kmeans_labels').mean()

print(cluster_profiles)

                latitude   longitude     stars  review_count  avg_review_stars
kmeans_labels                                                                 
0              34.425606 -119.715876  3.943820     76.011236          3.963505
1              34.425788 -119.707387  4.178571    245.928571          4.205683
2              34.425520 -119.710181  3.945017     15.958763          3.922660


In [9]:
# Calculate the statistical summary for each cluster
cluster_profiles = df.groupby('kmeans_labels').describe()

print(cluster_profiles)

              latitude                                                        \
                 count       mean       std        min        25%        50%   
kmeans_labels                                                                  
0                 89.0  34.425606  0.009702  34.401619  34.419242  34.421798   
1                 14.0  34.425788  0.013045  34.410014  34.417326  34.421016   
2                582.0  34.425520  0.011712  34.401144  34.419236  34.421781   

                                    longitude              ... review_count  \
                     75%        max     count        mean  ...          75%   
kmeans_labels                                              ...                
0              34.436396  34.443643      89.0 -119.715876  ...        89.00   
1              34.435605  34.456488      14.0 -119.707387  ...       270.75   
2              34.430488  34.532293     582.0 -119.710181  ...        22.00   

                     avg_review_stars       

In [12]:
import sqlite3
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import folium

# Connect to the SQLite database
conn = sqlite3.connect('D:\Coursera\Files\Files\home\coder\project\master.db')

# Define the SQL query
query = """
SELECT b.latitude, b.longitude, b.stars, COUNT(r.review_id) as review_count, AVG(r.stars) as avg_review_stars,
       AVG(u.useful) as avg_useful, AVG(u.funny) as avg_funny, AVG(u.cool) as avg_cool, AVG(u.fans) as avg_fans, AVG(u.average_stars) as avg_user_stars
FROM business b
JOIN review r ON b.business_id = r.business_id
JOIN user u ON r.user_id = u.user_id
WHERE b.city = 'Santa Barbara' 
AND b.state = 'CA' 
AND b.categories LIKE '%Shopping%'
AND b.is_open = 1
GROUP BY b.business_id
"""

# Execute the query and load the results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Define the clustering algorithm
kmeans = KMeans(n_clusters=5)

# Apply the clustering algorithm to the data
features = df[['latitude', 'longitude', 'stars', 'review_count', 'avg_review_stars', 'avg_useful', 'avg_funny', 'avg_cool', 'avg_fans', 'avg_user_stars']]
df['kmeans_labels'] = kmeans.fit_predict(features)

# Calculate the silhouette score
silhouette = silhouette_score(features, df['kmeans_labels'])
print(f'Silhouette Score: {silhouette}')

# Define the colors for the clusters
colors = ['red', 'blue', 'green', 'purple', 'orange']

# Plot the clusters on the map
m = folium.Map(location=[34.4213, -119.6982], zoom_start=13)

for index, row in df.iterrows():
    folium.Marker([row['latitude'], row['longitude']], 
                  icon=folium.Icon(color=colors[int(row['kmeans_labels'])]), 
                  popup=f"Stars: {row['stars']}, Review Count: {row['review_count']}, Average Review Stars: {row['avg_review_stars']}, Useful: {row['avg_useful']}, Funny: {row['avg_funny']}, Cool: {row['avg_cool']}, Fans: {row['avg_fans']}, User Stars: {row['avg_user_stars']}").add_to(m)

m.save(r'D:\Coursera\Files\Files\home\coder\project\kmeans_map_silhouette_score.html')


 # métrica de silhueta, que é uma medida de quão semelhante um objeto é ao seu próprio cluster em comparação com outros clusters. O valor da métrica de silhueta varia de -1 a 1, onde um valor alto indica que o objeto está bem combinado com seu próprio cluster e mal combinado com os clusters vizinhos.

c:\Users\surze\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


Silhouette Score: 0.7759572811050639


In [13]:
# Calculate the mean of each feature for each cluster
cluster_profiles = df.groupby('kmeans_labels').mean()

print(cluster_profiles)

                latitude   longitude     stars  review_count  \
kmeans_labels                                                  
0              34.426834 -119.712977  4.052696     30.882353   
1              34.420627 -119.704062  3.961538     34.384615   
2              34.420645 -119.720804  4.750000     13.500000   
3              34.423731 -119.708692  3.910000     37.460000   
4              34.417274 -119.677245  4.000000     57.333333   

               avg_review_stars   avg_useful    avg_funny     avg_cool  \
kmeans_labels                                                            
0                      4.033205   117.362304    45.278234    68.843391   
1                      3.955540  2770.430892  1446.242068  2216.519197   
2                      4.704545  7833.345455  4855.740909  7408.095455   
3                      3.938884  1027.927079   530.373463   802.412945   
4                      3.993263  5300.981654  3351.359567  4435.928361   

                 avg_fans  avg_u

In [14]:
import sqlite3
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import folium

# Connect to the SQLite database
conn = sqlite3.connect('D:\Coursera\Files\Files\home\coder\project\master.db')

# Define the SQL query
query = """
SELECT b.latitude, b.longitude, b.stars, COUNT(r.review_id) as review_count, AVG(r.stars) as avg_review_stars,
       AVG(u.useful) as avg_useful, AVG(u.funny) as avg_funny, AVG(u.cool) as avg_cool, AVG(u.fans) as avg_fans, AVG(u.average_stars) as avg_user_stars
FROM business b
JOIN review r ON b.business_id = r.business_id
JOIN user u ON r.user_id = u.user_id
WHERE b.city = 'Santa Barbara' 
AND b.state = 'CA' 
AND b.categories LIKE '%Shopping%'
AND b.is_open = 1
GROUP BY b.business_id
"""

# Execute the query and load the results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Define the clustering algorithm
kmeans = KMeans(n_clusters=4)

# Apply the clustering algorithm to the data
features = df[['latitude', 'longitude', 'stars', 'review_count', 'avg_review_stars', 'avg_useful', 'avg_funny', 'avg_cool', 'avg_fans', 'avg_user_stars']]
df['kmeans_labels'] = kmeans.fit_predict(features)

# Calculate the silhouette score
silhouette = silhouette_score(features, df['kmeans_labels'])
print(f'Silhouette Score: {silhouette}')

# Define the colors for the clusters
colors = ['red', 'blue', 'green', 'orange']

# Plot the clusters on the map
m = folium.Map(location=[34.4213, -119.6982], zoom_start=13)

for index, row in df.iterrows():
    folium.Marker([row['latitude'], row['longitude']], 
                  icon=folium.Icon(color=colors[int(row['kmeans_labels'])]), 
                  popup=f"Cluster: {row['kmeans_labels']}, Stars: {row['stars']}, Review Count: {row['review_count']}, Average Review Stars: {row['avg_review_stars']}, Useful: {row['avg_useful']}, Funny: {row['avg_funny']}, Cool: {row['avg_cool']}, Fans: {row['avg_fans']}, User Stars: {row['avg_user_stars']}").add_to(m)

m.save(r'D:\Coursera\Files\Files\home\coder\project\kmeans_map.html')

c:\Users\surze\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


Silhouette Score: 0.7653109863493083


In [20]:
import sqlite3
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import folium

# Connect to the SQLite database
conn = sqlite3.connect('D:\Coursera\Files\Files\home\coder\project\master.db')

# Define the SQL query
query = """
SELECT b.latitude, b.longitude, b.stars, COUNT(r.review_id) as review_count, AVG(r.stars) as avg_review_stars,
       AVG(u.useful) as avg_useful, AVG(u.funny) as avg_funny, AVG(u.cool) as avg_cool, AVG(u.fans) as avg_fans, AVG(u.average_stars) as avg_user_stars
FROM business b
JOIN review r ON b.business_id = r.business_id
JOIN user u ON r.user_id = u.user_id
WHERE b.city = 'Santa Barbara' 
AND b.state = 'CA' 
AND b.categories LIKE '%Shopping%'
AND b.is_open = 1
GROUP BY b.business_id
"""

# Execute the query and load the results into a pandas DataFrame
df = pd.read_sql_query(query, conn)

# Define the clustering algorithm
kmeans = KMeans(n_clusters=3)

# Apply the clustering algorithm to the data
features = df[['latitude', 'longitude', 'stars', 'review_count', 'avg_review_stars', 'avg_useful', 'avg_funny', 'avg_cool', 'avg_fans', 'avg_user_stars']]
df['kmeans_labels'] = kmeans.fit_predict(features)

# Calculate the silhouette score
silhouette = silhouette_score(features, df['kmeans_labels'])
print(f'Silhouette Score: {silhouette}')

# Define the colors for the clusters
colors = ['red', 'blue', 'green']

# Plot the clusters on the map
m = folium.Map(location=[34.4213, -119.6982], zoom_start=13)

for index, row in df.iterrows():
    folium.Marker([row['latitude'], row['longitude']], 
                  icon=folium.Icon(color=colors[int(row['kmeans_labels'])]), 
                  popup=f"Cluster: {row['kmeans_labels']}, Stars: {row['stars']}, Review Count: {row['review_count']}, Average Review Stars: {row['avg_review_stars']}, Useful: {row['avg_useful']}, Funny: {row['avg_funny']}, Cool: {row['avg_cool']}, Fans: {row['avg_fans']}, User Stars: {row['avg_user_stars']}").add_to(m)

m.save(r'D:\Coursera\Files\Files\home\coder\project\kmeans_map.html')

c:\Users\surze\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


Silhouette Score: 0.8455447357855373


In [21]:
# Group by 'kmeans_labels', calculate the mean, and drop 'latitude' and 'longitude' columns
cluster_profiles = df.groupby('kmeans_labels').mean().drop(['latitude', 'longitude'], axis=1)
cluster_profiles

,stars,review_count,avg_review_stars,avg_useful,avg_funny,avg_cool,avg_fans,avg_user_stars
kmeans_labels,,,,,,,,
0,4.050676,30.916667,4.034133,176.939772,75.313416,116.614019,6.299834,3.938751
1,3.837500,41.000000,3.854520,2318.752964,1228.883239,1851.540644,66.703958,3.890363
2,4.187500,46.375000,4.171083,5934.072604,3727.454902,5178.970134,158.722745,4.007560


In [22]:
# Count the number of cases per cluster
cluster_counts = df.groupby('kmeans_labels').size()

print(cluster_counts)

kmeans_labels
0    444
1     40
2      8
dtype: int64


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 492 entries, 0 to 491
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   latitude          492 non-null    float64
 1   longitude         492 non-null    float64
 2   stars             492 non-null    float64
 3   review_count      492 non-null    int64  
 4   avg_review_stars  492 non-null    float64
 5   avg_useful        492 non-null    float64
 6   avg_funny         492 non-null    float64
 7   avg_cool          492 non-null    float64
 8   avg_fans          492 non-null    float64
 9   avg_user_stars    492 non-null    float64
 10  kmeans_labels     492 non-null    int32  
dtypes: float64(9), int32(1), int64(1)
memory usage: 40.5 KB
